In [16]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [17]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mi.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,4,4,4,4,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN
1,8,8,8,8,8,9725917702,5/1/20,101,PPP,POPPYCOCKS INC,...,999 Other,4442 Lawn and Garden Equipment and Supplies St...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,NaN
2,10,10,10,10,10,9662697710,5/1/20,101,PPP,"MILFAST INDUSTRIAL SUPPLY, INC",...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,NaN
3,449017,449017,449017,449017,449017,1752647202,4/15/20,515,PPP,"ADVANTAGE CONSULTING AND EDUCATIONAL SERVICES,...",...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14
4,449018,449018,449018,449018,449018,2139978210,7/31/20,515,PPP,"MORLEY COMPANIES, INCORPORATED",...,999 Other,4543 Direct Selling Establishments,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,478818,478818,478818,478818,478818,9646208307,1/31/21,515,PPS,SHORELINE PEDIATRICS PLC,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,NaN
29805,478819,478819,478819,478819,478819,9648118308,1/31/21,515,PPS,EPCO SALES LLC,...,332 Fabricated Metal Products,3325 Hardware Manufacturing,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,NaN
29806,478820,478820,478820,478820,478820,9689138310,1/31/21,515,PPS,MICHIGAN MEDICAL GROUP P.C.,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,NaN
29807,478821,478821,478821,478821,478821,9788397208,4/28/20,515,PPP,ADRIAN PRECISION MACHINING LLC,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,NaN


In [18]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [19]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long
5478,8205248602,"5000 N US ROUTE 131, Reed City, MI",NaN,NaN
5521,9918728402,"3741 Roger B Chaffee Mem Dr SE, Grand Rapids, MI",NaN,NaN
7123,9074147005,"5000 N US 131, REED CITY, MI",NaN,NaN
8582,7726897004,"2547 3 MILE RD NW Ste F, GRAND RAPIDS, MI",NaN,NaN
8595,8573248301,"3760 John F Donnelly Dr, Holland, MI",NaN,NaN
11666,5836287000,"612 S Park Street, KALAMAZOO, MI",NaN,NaN
11672,7908347102,"3602 WESTNEDGE AVE, KALAMAZOO, MI",NaN,NaN
15967,5396147010,"250 AMES ST, ELK RAPIDS, MI",NaN,NaN
16867,7855317001,"51194 Romeo Plank Rd Ste 257, MACOMB, MI",NaN,NaN
20303,9059797308,"14471 W 11 MILE RD, OAK PARK, MI",NaN,NaN


In [20]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,9662437702,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680
1,9725917702,"128 E Front St, Traverse City, MI",44.763818,-85.622327
2,9662697710,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168
3,1752647202,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390
4,2139978210,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [12]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MI_fips_scraped.csv")


In [ ]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MI_fips_scraped.csv") 
fips_check_scraped.head() 

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [57]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [5]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [6]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,3506337207,"238 COURTHOUSE RD, GULFPORT, MS",30.381123,-89.044000,2.804700e+14
1,8685327005,"1020 HIGHLAND COLONY PKWY, RIDGELAND, MS",32.446636,-90.151970,2.808903e+14
2,5587558700,"109 S Broadway St, Tupelo, MS",34.257282,-88.705376,2.808195e+14
3,8446217010,"4374-A MANGUM DR, FLOWOOD, MS",32.319440,-90.107650,2.812102e+14
4,3918387109,"600 FRANKLIN DR, HOUSTON, MS",33.914646,-89.016710,2.801795e+14
...,...,...,...,...,...
5497,6662018610,"300 Heritage Dr, Oxford, MS",34.367306,-89.560310,2.807195e+14
5498,7192318409,"6229 Highway 305 N Ste O, Olive Branch, MS",34.952320,-89.830414,2.803307e+14
5499,7483017308,"41033 OLD HIGHWAY 45 South, ABERDEEN, MS",33.810067,-88.477517,2.809595e+14
5500,7726238600,"116 Price St, McComb, MS",31.247960,-90.469820,2.811395e+14


In [7]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

LoanNumber    18
full_add      18
Lat           18
Long          18
FIPS_z         0
dtype: int64

<h4>FIPS Script

In [22]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [35]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('26')] #MI
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


  0%|          | 0/29795 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [36]:
geo_id_list =geo_id 

Length Check

In [37]:
print(len(geo_id_list))

29795


Matching FIPS to DataFrame

In [38]:
#for geocode file
fips_fill = fips_check_notna
#if FIPS available
#fips_fill = fips_check_scraped_null.iloc[:1210]

In [39]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_886/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,full_add,Lat,Long,FIPS_z
0,9662437702,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,[260992551002000]
1,9725917702,"128 E Front St, Traverse City, MI",44.763818,-85.622327,[260555512001017]
2,9662697710,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,[260992239022000]
3,1752647202,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,[261251616002002]
4,2139978210,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,[261450004001006]
...,...,...,...,...,...
29804,9646208307,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,[261210001005000]
29805,9648118308,"601 Kelso St, Flint, MI",43.020582,-83.651218,[260490031002006]
29806,9689138310,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,[261635836002014]
29807,9788397208,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,[260910617023013]


Remove brackets

In [40]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/t9/c5y08h4d40j0xvl66n01j2680000gn/T/ipykernel_886/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,full_add,Lat,Long,FIPS_z
0,9662437702,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,260992551002000
1,9725917702,"128 E Front St, Traverse City, MI",44.763818,-85.622327,260555512001017
2,9662697710,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,260992239022000
3,1752647202,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,261251616002002
4,2139978210,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,261450004001006
...,...,...,...,...,...
29804,9646208307,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,261210001005000
29805,9648118308,"601 Kelso St, Flint, MI",43.020582,-83.651218,260490031002006
29806,9689138310,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,261635836002014
29807,9788397208,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,260910617023013


In [41]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
457,8395967305,"200 N. Park 0.0, Kalamazoo, MI",48.025208,-98.127360,NaN
769,5652057007,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN
1490,9658427010,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN
1787,4120198309,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
10203,5833567104,"666 E MAIN ST Suite A-2, CENTREVILLE, MI",39.958540,-82.981387,NaN
10823,7075478401,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN
11790,1919668403,"5665W Hwy US 2, Manistique, MI",48.354809,-114.238664,NaN
11791,2703197109,"5665 W HIGHWAY US 2, MANISTIQUE, MI",48.354809,-114.238664,NaN
12483,3187267103,"9067 US Highway 31, BERRIEN SPRINGS, MI",34.109378,-86.809378,NaN
16063,1405218302,"6279 US Highway 31 S, Charlevoix, MI",39.386384,-85.985616,NaN


Parse for Merge

In [42]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,9662437702,260992551002000
1,9725917702,260555512001017
2,9662697710,260992239022000
3,1752647202,261251616002002
4,2139978210,261450004001006
...,...,...
29804,9646208307,261210001005000
29805,9648118308,260490031002006
29806,9689138310,261635836002014
29807,9788397208,260910617023013


Read in base data

In [43]:
#if from geocode file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_mi.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

#if from FIPS file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MI_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
#fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,4,4,4,4,4,4,4,4,4,4,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN
1,8,8,8,8,8,8,8,8,8,8,...,999 Other,4442 Lawn and Garden Equipment and Supplies St...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,NaN
2,10,10,10,10,10,10,10,10,10,10,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,NaN
3,449017,449017,449017,449017,449017,449017,449017,449017,449017,449017,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14
4,449018,449018,449018,449018,449018,449018,449018,449018,449018,449018,...,999 Other,4543 Direct Selling Establishments,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,478818,478818,478818,478818,478818,478818,478818,478818,478818,478818,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,NaN
29805,478819,478819,478819,478819,478819,478819,478819,478819,478819,478819,...,332 Fabricated Metal Products,3325 Hardware Manufacturing,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,NaN
29806,478820,478820,478820,478820,478820,478820,478820,478820,478820,478820,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,NaN
29807,478821,478821,478821,478821,478821,478821,478821,478821,478821,478821,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,NaN


Merge with MainFrame

In [44]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,4,4,4,4,4,4,4,4,4,4,...,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,260992551002000
1,8,8,8,8,8,8,8,8,8,8,...,4442 Lawn and Garden Equipment and Supplies St...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,NaN,260555512001017
2,10,10,10,10,10,10,10,10,10,10,...,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,NaN,260992239022000
3,449017,449017,449017,449017,449017,449017,449017,449017,449017,449017,...,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14,261251616002002
4,449018,449018,449018,449018,449018,449018,449018,449018,449018,449018,...,4543 Direct Selling Establishments,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,NaN,261450004001006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,478818,478818,478818,478818,478818,478818,478818,478818,478818,478818,...,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,NaN,261210001005000
29805,478819,478819,478819,478819,478819,478819,478819,478819,478819,478819,...,3325 Hardware Manufacturing,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,NaN,260490031002006
29806,478820,478820,478820,478820,478820,478820,478820,478820,478820,478820,...,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,NaN,261635836002014
29807,478821,478821,478821,478821,478821,478821,478821,478821,478821,478821,...,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,NaN,260910617023013


<b>Fill FIPS Columns - if FIPS file

In [45]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,4,4,4,4,4,4,4,4,4,4,...,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,260992551002000,260992551002000
1,8,8,8,8,8,8,8,8,8,8,...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,NaN,260555512001017,260555512001017
2,10,10,10,10,10,10,10,10,10,10,...,4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,NaN,260992239022000,260992239022000
3,449017,449017,449017,449017,449017,449017,449017,449017,449017,449017,...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14,261251616002002,261251616002002
4,449018,449018,449018,449018,449018,449018,449018,449018,449018,449018,...,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,NaN,261450004001006,261450004001006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,478818,478818,478818,478818,478818,478818,478818,478818,478818,478818,...,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,NaN,261210001005000,261210001005000
29805,478819,478819,478819,478819,478819,478819,478819,478819,478819,478819,...,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,NaN,260490031002006,260490031002006
29806,478820,478820,478820,478820,478820,478820,478820,478820,478820,478820,...,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,NaN,261635836002014,261635836002014
29807,478821,478821,478821,478821,478821,478821,478821,478821,478821,478821,...,3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,NaN,260910617023013,260910617023013


Drop Extra FIPS Columns

In [46]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,4,4,4,4,4,4,4,4,4,4,...,999 Other,4842 Specialized Freight Trucking,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,260992551002000
1,8,8,8,8,8,8,8,8,8,8,...,999 Other,4442 Lawn and Garden Equipment and Supplies St...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,260555512001017
2,10,10,10,10,10,10,10,10,10,10,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,260992239022000
3,449017,449017,449017,449017,449017,449017,449017,449017,449017,449017,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,261251616002002
4,449018,449018,449018,449018,449018,449018,449018,449018,449018,449018,...,999 Other,4543 Direct Selling Establishments,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,261450004001006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,478818,478818,478818,478818,478818,478818,478818,478818,478818,478818,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,261210001005000
29805,478819,478819,478819,478819,478819,478819,478819,478819,478819,478819,...,332 Fabricated Metal Products,3325 Hardware Manufacturing,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,260490031002006
29806,478820,478820,478820,478820,478820,478820,478820,478820,478820,478820,...,999 Other,6211 Offices of Physicians,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,261635836002014
29807,478821,478821,478821,478821,478821,478821,478821,478821,478821,478821,...,332 Fabricated Metal Products,"3327 Machine Shops; Turned Product; and Screw,...",3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,260910617023013


In [50]:
fips_merge_drop = fips_merge.drop(["Unnamed: 0.1","Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
769,449783,5652057007,4/6/20,515,PPP,UPPER GREAT LAKES FAMILY HEALTH CENTER,135 STATE HIGHWAY M35,GWINN,MI,49841.0,...,6211.0,MICHIGAN DISTRICT OFFICE,d $2-5 million,100 to 249,"135 STATE HIGHWAY M35, GWINN, MI",48.024810,-98.124330,NaN,NaN,NaN
1490,450504,9658427010,4/9/20,515,PPP,"GONGOS, INC.",150 W. 2nd Street Suite 300 0.0,Royal Oak,MI,48067.0,...,5419.0,MICHIGAN DISTRICT OFFICE,c $1-2 million,100 to 249,"150 W. 2nd Street Suite 300 0.0, Royal Oak, MI",48.025192,-98.123570,NaN,NaN,NaN
1787,450801,4120198309,1/22/21,515,PPS,MAGNUM HOSPITALITY INC.,5112 US Highway 31 N,Williamsburg,MI,49690.0,...,7225.0,MICHIGAN DISTRICT OFFICE,c $1-2 million,50 to 99,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN,NaN,NaN
5478,454492,8205248602,3/24/21,515,PPS,UUSI LLC,5000 N US ROUTE 131,Reed City,MI,49677.0,...,3344.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"5000 N US ROUTE 131, Reed City, MI",NaN,NaN,NaN,NaN,NaN
5521,454535,9918728402,2/18/21,515,PPS,HOEKSTRA TRANSPORTATION INC.,3741 Roger B Chaffee Mem Dr SE,Grand Rapids,MI,49548.0,...,4413.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"3741 Roger B Chaffee Mem Dr SE, Grand Rapids, MI",NaN,NaN,NaN,NaN,NaN
7123,456137,9074147005,4/9/20,515,PPP,UUSI LLC,5000 N US 131,REED CITY,MI,49677.0,...,3369.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",50 to 99,"5000 N US 131, REED CITY, MI",NaN,NaN,NaN,NaN,NaN
8582,457596,7726897004,4/8/20,515,PPP,MIEN COMPANY,2547 3 MILE RD NW Ste F,GRAND RAPIDS,MI,49534.0,...,4539.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"2547 3 MILE RD NW Ste F, GRAND RAPIDS, MI",NaN,NaN,NaN,NaN,NaN
8595,457609,8573248301,1/29/21,515,PPS,TRIC TOOL LTD,3760 John F Donnelly Dr,Holland,MI,49424.0,...,3335.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"3760 John F Donnelly Dr, Holland, MI",NaN,NaN,NaN,NaN,NaN
10203,459217,5833567104,4/14/20,515,PPP,INVESTMENT PROPERTY ADVISORS LLC,666 E MAIN ST Suite A-2,CENTREVILLE,MI,49032.0,...,5313.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"666 E MAIN ST Suite A-2, CENTREVILLE, MI",39.958540,-82.981387,NaN,NaN,NaN
10823,459837,7075478401,2/11/21,515,PPS,CLAM-TORCH RMPL INC.,5112 US Highway 31 N,Williamsburg,MI,49690.0,...,7224.0,MICHIGAN DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"5112 US Highway 31 N, Williamsburg, MI",39.402598,-85.993189,NaN,NaN,NaN


Convert to File

In [51]:
fips_MI = fips_merge_drop


In [52]:
fips_MI.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MI_fips_scraped.csv") 

Review & Compare

In [53]:
fips_MI = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/MI_fips_scraped.csv") 
fips_MI

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,0,4,9662437702,5/1/20,101,PPP,AERO BOX LLC,20101 Cornillie Dr,Roseville,MI,...,4842.0,MASSACHUSETTS DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"20101 Cornillie Dr, Roseville, MI",42.521828,-82.913680,NaN,2.609926e+14,2.609926e+14
1,1,8,9725917702,5/1/20,101,PPP,POPPYCOCKS INC,128 E Front St,Traverse City,MI,...,4442.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"128 E Front St, Traverse City, MI",44.763818,-85.622327,NaN,2.605555e+14,2.605555e+14
2,2,10,9662697710,5/1/20,101,PPP,"MILFAST INDUSTRIAL SUPPLY, INC",51475 Milano Dr,Macomb County,MI,...,4238.0,MASSACHUSETTS DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"51475 Milano Dr, Macomb County, MI",42.674303,-82.967168,NaN,2.609922e+14,2.609922e+14
3,3,449017,1752647202,4/15/20,515,PPP,"ADVANTAGE CONSULTING AND EDUCATIONAL SERVICES,...","25800 Northwestern Highway, Suite 720",SOUTHFIELD,MI,...,6231.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"25800 Northwestern Highway, Suite 720, SOUTHFI...",42.478592,-83.249390,2.612516e+14,2.612516e+14,2.612516e+14
4,4,449018,2139978210,7/31/20,515,PPP,"MORLEY COMPANIES, INCORPORATED",1 Morley Plaza,SAGINAW,MI,...,4543.0,MICHIGAN DISTRICT OFFICE,e $5-10 million,500 or more,"1 Morley Plaza, SAGINAW, MI",43.434079,-83.938940,NaN,2.614500e+14,2.614500e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29804,29804,478818,9646208307,1/31/21,515,PPS,SHORELINE PEDIATRICS PLC,684 Harvey St Ste 201,Muskegon,MI,...,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"684 Harvey St Ste 201, Muskegon, MI",43.239902,-86.203271,NaN,2.612100e+14,2.612100e+14
29805,29805,478819,9648118308,1/31/21,515,PPS,EPCO SALES LLC,601 Kelso St,Flint,MI,...,3325.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"601 Kelso St, Flint, MI",43.020582,-83.651218,NaN,2.604900e+14,2.604900e+14
29806,29806,478820,9689138310,1/31/21,515,PPS,MICHIGAN MEDICAL GROUP P.C.,22348 Wick Rd,Taylor,MI,...,6211.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"22348 Wick Rd, Taylor, MI",42.241625,-83.254483,NaN,2.616358e+14,2.616358e+14
29807,29807,478821,9788397208,4/28/20,515,PPP,ADRIAN PRECISION MACHINING LLC,605 INDUSTRIAL DR,ADRIAN,MI,...,3327.0,MICHIGAN DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"605 INDUSTRIAL DR, ADRIAN, MI",41.889386,-84.064932,NaN,2.609106e+14,2.609106e+14
